先切分数据为训练集和测试集

In [1]:
import json
datas=[]
with open('aime_amc_new.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        data['category']='aime_amc'
        datas.append(data)
datas2=[]
with open('minif2f.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        data['category']='minif2f'
        datas2.append(data)
with open('MATH.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        data['category']='MATH'
        datas2.append(data)

{'name': 'ToT_19',
 'split': 'MATH',
 'informal_prefix': 'Consider two positive integers $a$ and $b$ such that $a^{n+1} + b^{n+1}$ is divisible by $a^n + b^n$ for infi\x0cnitely many positive integers $n$. Is it necessarily true that $a = b$?\n\n(Boris Frenkin)\n Answer: $   \\text { No }  $\n',
 'formal_statement': 'theorem frenkin_problem :\n  ¬ ∀ (a b : ℕ) (h_pos_a : a > 0) (h_pos_b : b > 0), \n    (∀ (n : ℕ) (h_pos_n : n > 0), \n      (a^(n+1) + b^(n+1)) % (a^n + b^n) = 0) → \n    a = b := by\n  -- The proof will use a counterexample to show the statement is false\n   \n',
 'goal': 'n > 0), \n      (a^(n+1) + b^(n+1)) % (a^n + b^n) = 0) → \n    a = b',
 'header': 'import Mathlib\n',
 'domain': ['Mathematics -> Algebra -> Intermediate Algebra -> Exponential Functions',
  'Mathematics -> Number Theory -> Divisibility -> Other'],
 'difficulty': 6.0,
 'problem': 'Consider two positive integers $a$ and $b$ such that $a^{n+1} + b^{n+1}$ is divisible by $a^n + b^n$ for infi\x0cnitely many

In [2]:
datas.extend(datas2)
test_name = []
with open('MATH_TEST.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        # if data['split']=='test':
        test_name.append(data['name'])
train = []
for item in datas:
    if item['name'] not in test_name:
        train.append(item)


In [3]:
with open('MATH_TRAIN.jsonl','w',encoding='utf-8') as f:
    for data in train:
        f.write(json.dumps(data,ensure_ascii=False)+'\n')

In [40]:
len(datas)

961

In [2]:
# 筛选出AIME和AMC的题目
aime = []
amc = []
for data in datas:
    if 'AIME' in data['name']:
        aime.append(data)
    if 'AMC' in data['name']:
        amc.append(data)
for data in datas2:
    if 'aime' in data['name'] and data['split']=='test':
        aime.append(data)
    if 'amc' in data['name'] and data['split']=='test':
        amc.append(data)
# 再从原始列表中移除这些题目
for data in aime:
    try:
        datas.remove(data)
    except:
        datas2.remove(data)
for data in amc:
    try:
        datas.remove(data)
    except:
        datas2.remove(data)

In [9]:
#构建测试集
# 首先选skip_list，随机选一半
import random
random.seed(0)

In [10]:
with open('new_dataset/our_Model_1024shot.json','r',encoding='utf-8') as f:
    our_Model=json.load(f)
    our_Model = [data for data in our_Model if data['success']["our_Model_1024shot"]]

In [11]:
# 从our_Model中选出一半题目
random.shuffle(our_Model)
our_Model=our_Model[:int(len(our_Model)/2)]
our_names = [data['name'] for data in our_Model]

In [12]:
print(len(our_Model))

51


In [13]:
AIME_TEST=[]
AMC_TEST=[]
TEST=[]
for data in aime:
    if data['name'] in our_names:
        AIME_TEST.append(data['name'])
        TEST.append(data)
for data in amc:
    if data['name'] in our_names:
        AMC_TEST.append(data['name'])
        TEST.append(data)
# 查看已经选了多少
print(len(AIME_TEST))
print(len(AMC_TEST))

8
43


In [14]:
# 从剩下的AMC和AIME中选取，直到AIME达到120，AMC达到240
random.shuffle(aime)
random.shuffle(amc)
for data in aime:
    if len(AIME_TEST)<120 and data['name'] not in AIME_TEST:
        AIME_TEST.append(data)
        TEST.append(data)
for data in amc:
    if len(AMC_TEST)<240 and data['name'] not in AMC_TEST:
        AMC_TEST.append(data)
        TEST.append(data)


In [15]:
GTEST=AIME_TEST+AMC_TEST
with open('proof_append/TEST.json','w',encoding='utf-8') as f:
    json.dump(GTEST,f,ensure_ascii=False,indent=4)

In [16]:
# 导出jsonl
with open('MATH_TEST.jsonl','w',encoding='utf-8') as f:
    for data in TEST:
        f.write(json.dumps(data,ensure_ascii=False)+'\n')